In [10]:
!rm -rf hugeCTR_saved_model_DeepFM
!mkdir hugeCTR_saved_model_DeepFM

config = """{
  "solver": {
    "lr_policy": "fixed",
    "display": 1000,
    "max_iter": 50000,
    "gpu": [0],
    "batchsize": 512,
    "snapshot": 10000,
    "snapshot_prefix": "./hugeCTR_saved_model_DeepFM",
    "eval_interval": 1000,
    "eval_batches": 1000
  },
  
  "optimizer": {
    "type": "Adam",
    "global_update": true,
    "adam_hparam": {
      "learning_rate": 0.001,
      "beta1": 0.9,
      "beta2": 0.999,
      "epsilon": 0.0000001
    }
  },

  "layers": [ 
    {
      "name": "data",
      "type": "Data",
    "source": "./data/hugeCTR/filelist.txt",
      "eval_source": "./data/hugeCTR/test_filelist.txt",
      "check": "None",
      "label": {
        "top": "label",
        "label_dim": 1
      },
      "dense": {
        "top": "dense",
        "dense_dim": 1
      },
      "sparse": [
        {
          "top": "data1",
          "type": "DistributedSlot",
          "max_feature_num_per_sample": 2,
          "slot_num": 2
        }        
      ]
    },

    {
      "name": "sparse_embedding1",
      "type": "DistributedSlotSparseEmbeddingHash",
      "bottom": "data1",
      "top": "sparse_embedding1",
      "sparse_embedding_hparam": {
        "max_vocabulary_size_per_gpu": 200000,
        "embedding_vec_size": 11,
        "combiner": 0
      }
    },

    {
      "name": "reshape1",
      "type": "Reshape",
      "bottom": "sparse_embedding1",
      "top": "reshape1",
      "leading_dim": 11
    },

    {
      "name": "slice1",
      "type": "Slice",
      "bottom": "reshape1",
      "ranges": [[0,10], [10,11]],
      "top": ["slice11", "slice12"]
    },

    {
      "name": "reshape2",
      "type": "Reshape",
      "bottom": "slice11",
      "top": "reshape2",
      "leading_dim": 22
    },

    {
      "name": "reshape3",
      "type": "Reshape",
      "bottom": "slice12",
      "top": "reshape3",
      "leading_dim": 26
    },

    {
      "name": "slice2",
      "type": "Slice",
      "bottom": "dense",
      "ranges": [[0,13], [0,13]],
      "top": ["slice21", "slice22"]
    },
    
    {
      "name": "multiply1",
      "type": "Multiply",
      "bottom": "slice21",
      "weight_dims": [13, 10],
      "top": "multiply1"
    },

    {
      "name": "multiply2",
      "type": "Multiply",
      "bottom": "slice22",
      "weight_dims": [13, 1],
      "top": "multiply2"
    },

    {
      "name": "concat1",
      "type": "Concat",
      "bottom": ["reshape2","multiply1"],
      "top": "concat1"
    },
    
    {
      "name": "slice3",
      "type": "Slice",
      "bottom": "concat1",
      "ranges": [[0,390], [0,390]],
      "top": ["slice31", "slice32"]
    },

    {
      "name": "fc1",
      "type": "InnerProduct",
      "bottom": "slice31",
      "top": "fc1",
       "fc_param": {
        "num_output": 400
      }
    },

    {
      "name": "relu1",
      "type": "ReLU",
      "bottom": "fc1",
      "top": "relu1" 
    },

    {
      "name": "dropout1",
      "type": "Dropout",
      "rate": 0.5,
      "bottom": "relu1",
      "top": "dropout1" 
    },

    {
      "name": "fc2",
      "type": "InnerProduct",
      "bottom": "dropout1",
      "top": "fc2",
       "fc_param": {
        "num_output": 400
      }
    },

    {
      "name": "relu2",
      "type": "ReLU",
      "bottom": "fc2",
      "top": "relu2"     
    },

    {
      "name": "dropout2",
      "type": "Dropout",
      "rate": 0.5,
      "bottom": "relu2",
      "top": "dropout2" 
    },

    {
      "name": "fc3",
      "type": "InnerProduct",
      "bottom": "dropout2",
      "top": "fc3",
       "fc_param": {
        "num_output": 400
      }
    },

    {
      "name": "relu3",
      "type": "ReLU",
      "bottom": "fc3",
      "top": "relu3"     
    },

    {
      "name": "dropout3",
      "type": "Dropout",
      "rate": 0.5,
      "bottom": "relu3",
      "top": "dropout3" 
    },

    {
      "name": "fc4",
      "type": "InnerProduct",
      "bottom": "dropout3",
      "top": "fc4",
       "fc_param": {
        "num_output": 1
      }
    },


    {
     "name": "fmorder2",
     "type": "FmOrder2",
     "bottom": "slice32",
     "out_dim": 10,
     "top": "fmorder2"
    },

    {
      "name": "reducesum1",
      "type": "ReduceSum",
      "bottom": "fmorder2",
      "axis": 1,
      "top": "reducesum1"
    },


    {
      "name": "concat2",
      "type": "Concat",
      "bottom": ["reshape3","multiply2"],
      "top": "concat2"
    },

    {
      "name": "reducesum2",
      "type": "ReduceSum",
      "bottom": "concat2",
      "axis": 1,
      "top": "reducesum2"
    },


    {
      "name": "add",
      "type": "Add",
      "bottom": ["fc4", "reducesum1", "reducesum2"],
      "top": "add"
    },
    
    {
      "name": "loss",
      "type": "BinaryCrossEntropyLoss",
      "bottom": ["add","label"],
      "top": "loss"
    } 
  ]
}"""

In [11]:
with open('./movie_lens_DeepFM.json', 'wt') as f:
    for line in config:
        f.write(line)

In [12]:
!../build/bin/huge_ctr --train ./movie_lens_DeepFM.json

[0, init_start, ]
HugeCTR Version: 2.2
Config file: ./movie_lens_DeepFM.json
[11d05h48m18s][HUGECTR][INFO]: batchsize_eval is not specified using default: 512
[11d05h48m18s][HUGECTR][INFO]: Default evaluation metric is AUC without threshold value
[11d05h48m18s][HUGECTR][INFO]: algorithm_search is not specified using default: 1
[11d05h48m18s][HUGECTR][INFO]: Algorithm search: ON
[11d05h48m20s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-16GB
[11d05h48m20s][HUGECTR][INFO]: Initial seed is 2185651270
[11d05h48m20s][HUGECTR][INFO]: cache_eval_data is not specified using default: 0
[11d05h48m20s][HUGECTR][INFO]: max_nnz is not specified using default: 2
[11d05h48m20s][HUGECTR][INFO]: num_internal_buffers 1
[11d05h48m20s][HUGECTR][INFO]: num_internal_buffers 1
[11d05h48m20s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=200000
[HCDEBUG][ERROR] Runtime error: n_in_elems % leading_dim != 0 /hugectr/HugeCTR/src/layers/reshape_layer.cu:81 

[HCDEBUG][ERR